## **Flights Data Exploration Challenge**

A significant part of a data scientist's role is to explore, analyze, and visualize data. In this challenge, you'll explore a real-world dataset containing flights data from the US Department of Transportation.

Let's start by loading the required packages.


In [1]:
# Load packages that you'll require in this adventure of R
suppressPackageStartupMessages({
  library(tidyverse)
  library(summarytools)
  library(glue)
  library(patchwork)
  })


In [ ]:
# A Blank cell

Now, we can import the data into R and start doing some data science on it!



In [2]:
# Load and view the data
df_flights <- read_csv("https://raw.githubusercontent.com/MicrosoftDocs/ml-basics/master/challenges/data/flights.csv", show_col_types = FALSE)

df_flights %>%
  slice_head(n = 7)


Year,Month,DayofMonth,DayOfWeek,Carrier,OriginAirportID,OriginAirportName,OriginCity,OriginState,DestAirportID,DestAirportName,DestCity,DestState,CRSDepTime,DepDelay,DepDel15,CRSArrTime,ArrDelay,ArrDel15,Cancelled
<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2013,9,16,1,DL,15304,Tampa International,Tampa,FL,12478,John F. Kennedy International,New York,NY,1539,4,0,1824,13,0,0
2013,9,23,1,WN,14122,Pittsburgh International,Pittsburgh,PA,13232,Chicago Midway International,Chicago,IL,710,3,0,740,22,1,0
2013,9,7,6,AS,14747,Seattle/Tacoma International,Seattle,WA,11278,Ronald Reagan Washington National,Washington,DC,810,-3,0,1614,-7,0,0
2013,7,22,1,OO,13930,Chicago O'Hare International,Chicago,IL,11042,Cleveland-Hopkins International,Cleveland,OH,804,35,1,1027,33,1,0
2013,5,16,4,DL,13931,Norfolk International,Norfolk,VA,10397,Hartsfield-Jackson Atlanta International,Atlanta,GA,545,-1,0,728,-9,0,0
2013,7,28,7,UA,12478,John F. Kennedy International,New York,NY,14771,San Francisco International,San Francisco,CA,1710,87,1,2035,183,1,0
2013,10,6,7,WN,13796,Metropolitan Oakland International,Oakland,CA,12191,William P Hobby,Houston,TX,630,-1,0,1210,-3,0,0


The dataset contains observations of US domestic flights in 2013, and consists of the following fields:

-   **Year**: The year of the flight (all records are from 2013)

-   **Month**: The month of the flight

-   **DayofMonth**: The day of the month on which the flight departed

-   **DayOfWeek**: The day of the week on which the flight departed - from 1 (Monday) to 7 (Sunday)

-   **Carrier**: The two-letter abbreviation for the airline.

-   **OriginAirportID**: A unique numeric identifier for the departure aiport

-   **OriginAirportName**: The full name of the departure airport

-   **OriginCity**: The departure airport city

-   **OriginState**: The departure airport state

-   **DestAirportID**: A unique numeric identifier for the destination aiport

-   **DestAirportName**: The full name of the destination airport

-   **DestCity**: The destination airport city

-   **DestState**: The destination airport state

-   **CRSDepTime**: The scheduled departure time

-   **DepDelay**: The number of minutes departure was delayed (flight that left ahead of schedule have a negative value)

-   **DelDelay15**: A binary indicator that departure was delayed by more than 15 minutes (and therefore considered "late")

-   **CRSArrTime**: The scheduled arrival time

-   **ArrDelay**: The number of minutes arrival was delayed (flight that arrived ahead of schedule have a negative value)

-   **ArrDelay15**: A binary indicator that arrival was delayed by more than 15 minutes (and therefore considered "late")

-   **Cancelled**: A binary indicator that the flight was cancelled

Your challenge is to explore the flight data to analyze possible factors that affect delays in departure or arrival of a flight.

1.  Start by cleaning the data.

    -   Identify any null or missing data, and impute appropriate replacement values.

    -   Identify and eliminate any outliers in the **DepDelay** and **ArrDelay** columns.

2.  Explore the cleaned data.

    -   View summary statistics for the numeric fields in the dataset.

    -   Determine the distribution of the **DepDelay** and **ArrDelay** columns.

    -   Use statistics, aggregate functions, and visualizations to answer the following questions:

        -   *What are the average (mean) departure and arrival delays?*

        -   *How do the carriers compare in terms of arrival delay performance?*

        -   *Is there a noticable difference in arrival delays for different days of the week?*

        -   *Which departure airport has the highest average departure delay?*

        -   *Do **late** departures tend to result in longer arrival delays than on-time departures?*

        -   *Which route (from origin airport to destination airport) has the most **late** arrivals?*

        -   *Which route has the highest average arrival delay?*

Sometimes, when we have a lot of columns in our data, it may difficult to get a grip of the data at first sight using `slice_head`

`glimpse` produces a transposed version where columns run down the page, and data runs across. This makes it possible to see every column in a data frame. Into the bargain, it also shows the dimension of the tibble and underlying data types of the columns.


In [3]:
# Get a glimpse of your data
df_flights %>%
  glimpse()


Rows: 271,940
Columns: 20
$ Year              <dbl> 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013…
$ Month             <dbl> 9, 9, 9, 7, 5, 7, 10, 7, 10, 5, 6, 7, 8, 7, 10, 4, 1…
$ DayofMonth        <dbl> 16, 23, 7, 22, 16, 28, 6, 28, 8, 12, 9, 21, 4, 17, 2…
$ DayOfWeek         <dbl> 1, 1, 6, 1, 4, 7, 7, 7, 2, 7, 7, 7, 7, 3, 7, 7, 4, 5…
$ Carrier           <chr> "DL", "WN", "AS", "OO", "DL", "UA", "WN", "EV", "AA"…
$ OriginAirportID   <dbl> 15304, 14122, 14747, 13930, 13931, 12478, 13796, 122…
$ OriginAirportName <chr> "Tampa International", "Pittsburgh International", "…
$ OriginCity        <chr> "Tampa", "Pittsburgh", "Seattle", "Chicago", "Norfol…
$ OriginState       <chr> "FL", "PA", "WA", "IL", "VA", "NY", "CA", "DC", "IL"…
$ DestAirportID     <dbl> 12478, 13232, 11278, 11042, 10397, 14771, 12191, 145…
$ DestAirportName   <chr> "John F. Kennedy International", "Chicago Midway Int…
$ DestCity          <chr> "New York", "Chicago", "Washington", "Cleveland", "A…
$ DestState   

## Clean missing values

Once you have imported your data, it is always a good idea to clean it. Sadly, this is often chronically underestimated, yet it's a fundamental step required for the subsequent operations in data analysis.

Let's find how many null values there are for each column.


In [4]:
# Find how many null values there are for each column.
colSums(is.na(df_flights))


Year             Month        DayofMonth         DayOfWeek 
                0                 0                 0                 0 
          Carrier   OriginAirportID OriginAirportName        OriginCity 
                0                 0                 0                 0 
      OriginState     DestAirportID   DestAirportName          DestCity 
                0                 0                 0                 0 
        DestState        CRSDepTime          DepDelay          DepDel15 
                0                 0                 0              2761 
       CRSArrTime          ArrDelay          ArrDel15         Cancelled 
                0                 0                 0                 0

Hmm, looks like there are some NA (missing values) `late departure` indicators (DepDel15 column). Departures are considered late if the delay is 15 minutes or more, so let's see the delays for the ones with an NA late indicator:

**Question 1.**

Starting with `df_flights`, **select** columns `DepDelay` and `DepDel15` then **filter** to obtain rows where the value of `DepDel15` is `NA`. Assign the results in a variable name `flights_depdel`.

Fill in the placeholder `....` with the right code.

In [ ]:
# Select columns DepDelay and DepDel15
# and then Filter the tibble to obtain
# observations where there is a missing value of DepDel15

flights_depdel <- df_flights %>%
  select(...., ....) %>%
  filter(is.na(....))


Test your answer:



In [ ]:
. <- ottr::check("tests/Question 1.R")


Good job! Now, let's `glimpse` at `flights_depdel`.



In [ ]:
flights_depdel %>%
  glimpse()


From the first few observations, it looks like the observations in `DepDel15` (a binary indicator that departure was delayed by more than 15 minutes) all have a corresponding delay of 0 in `DepDelay`(the number of minutes departure was delayed). Let's check by looking at the summary statistics for the `DepDelay` records:



In [ ]:
# Get summary statistics using summary function
df_flights %>%
  filter(rowSums(is.na(.)) > 0) %>%
  select(DepDelay) %>%
  summary()


The min, max, and mean are all 0; so it seems that none of these were actually *late* departures.

**Question 2.**

Starting with `df_flights`, replace the missing values in the column **DepDel15** with a `0`. Assign this to a variable name `df_flights`.

Fill in the placeholder `....` with the right code.

In [ ]:
# Replace missing values in DepDel15 with 0
df_flights <- df_flights %>%
  mutate(DepDel15 = ....)


Test your answer:



In [ ]:
. <- ottr::check("tests/Question 2.R")


Good job! No missing values now. Let's take this a little further.

### **Clean outliers**

An outlier is a data point that differs significantly from other observations. Let's create a function that shows the distribution and summary statistics for a given column.


In [ ]:
# Function to show summary stats and distribution for a column
show_distribution <- function(var_data, binwidth) {

  # Get summary statistics by first extracting values from the column
  min_val <- min(pull(var_data))
  max_val <- max(pull(var_data))
  mean_val <- mean(pull(var_data))
  med_val <- median(pull(var_data))
  mod_val <- statip::mfv(pull(var_data))

  # Print the stats
  stats <- glue::glue(
  "Minimum: {format(round(min_val, 2), nsmall = 2)}
   Mean: {format(round(mean_val, 2), nsmall = 2)}
   Median: {format(round(med_val, 2), nsmall = 2)}
   Mode: {format(round(mod_val, 2), nsmall = 2)}
   Maximum: {format(round(max_val, 2), nsmall = 2)}"
  )

  theme_set(theme_light())
  # Plot the histogram
  hist_gram <- ggplot(var_data) +
  geom_histogram(aes(x = pull(var_data)), binwidth = binwidth,
                 fill = "midnightblue", alpha = 0.7, boundary = 0.4) +

  # Add lines for the statistics
  geom_vline(xintercept = min_val, color = "gray33",
 linetype = "dashed", size = 1.3) +
  geom_vline(xintercept = mean_val, color = "cyan",
 linetype = "dashed", size = 1.3) +
  geom_vline(xintercept = med_val, color = "red",
 linetype = "dashed", size = 1.3) +
  geom_vline(xintercept = mod_val, color = "yellow",
 linetype = "dashed", size = 1.3) +
  geom_vline(xintercept = max_val, color = "gray33",
 linetype = "dashed", size = 1.3) +

  # Add titles and labels
  ggtitle("Data Distribution") +
  xlab("") +
  ylab("Frequency") +
  theme(plot.title = element_text(hjust = 0.5))

  # Plot the box plot
  bx_plt <- ggplot(data = var_data) +
  geom_boxplot(mapping = aes(x = pull(var_data), y = 1),
               fill = "#E69F00", color = "gray23", alpha = 0.7) +

    # Add titles and labels
  xlab("Value") +
  ylab("") +
  theme(plot.title = element_text(hjust = 0.5))


  # To return multiple outputs, use a `list`
  return(

    list(stats,
         hist_gram / bx_plt)) # End of returned outputs

} # End of function


**Question 3.** Starting with the `df_flights` data, only keep the **`DepDelay`** column. Assign this to a variable name `df_col`.

Once you have this figured out, call the function `show_distribution` with the arguments names and corresponding values as follows: `var_data = df_col` and `binwidth = 100`

From the function output, what's the distribution of **DepDelay** (the number of minutes departure was delayed)?

Fill in the placeholder `....` with the right code.

In [ ]:
# Select DepDelay column
df_col <- df_flights %>%
  ....

# Call the function show_distribution
show_distribution(var_data = df_col, binwidth = 100)


Test your answer:



In [ ]:
. <- ottr::check("tests/Question 3.R")


Now, let's investigate the distribution of **ArrDelay** (the number of minutes arrival was delayed)

**Question 4.** Starting with the `df_flights` data, only keep the **`ArrDelay`** column. Assign this to a variable name `df_col`.

Once you have this figured out, call the function `show_distribution` with the arguments names and corresponding values as follows: `var_data = df_col` and `binwidth = 100` (value of the width of each bin along the X-axis)

From the function output, what's the distribution of **ArrDelay**?

Fill in the placeholder `....` with the right code.

In [ ]:
# Select DepDelay column
df_col <- df_flights %>%
  ....

# Call the function show_distribution
show_distribution(var_data = df_col, binwidth = 100)


Test your answer:



In [ ]:
. <- ottr::check("tests/Question 4.R")


From both outputs, there are outliers at the lower and upper ends of both variables. Let's trim the data so that we include only rows where the values for these fields are within the 1st and 90th percentile. We begin with the **ArrDelay** observation.



In [ ]:
# Trim outliers for ArrDelay based on 1% and 90% percentiles
# Produce quantiles corresponding to 1% and 90%
arrdelay_01pcntile <- df_flights %>%
  pull(ArrDelay) %>%
  quantile(probs = 1 / 100, names = FALSE)

arrdelay_90pcntile <- df_flights %>%
  pull(ArrDelay) %>%
  quantile(probs = 90 / 100, names = FALSE)

# Print 1st and 90th quantiles respectively
cat(arrdelay_01pcntile, "\n", arrdelay_90pcntile)


Now that we have quantiles corresponding to 1% and 90%, let's filter the `df_flights` data to only include rows whose Arrival delay falls within this range.

**Question 5.** Starting with the `df_flights` data, filter to only include rows whose **ArrDelay** falls within 1st and 90th quantiles. Assign this to a variable name `df_flights`.

Fill in the placeholder `....` with the right code.

In [ ]:
# Filter data to remove outliers
df_flights <- df_flights %>%
  filter(ArrDelay > ...., ....)


Test your answer:



In [ ]:
. <- ottr::check("tests/Question 5.R")


Now, let's do the same for `DepDelay` column.

**Question 6.** Starting with the `df_flights` data, obtain quantiles corresponding to 1% and 90%. Assign these values to the variable names `depdelay_01pcntile` and `depdelay_90pcntile` respectively.

Fill in the placeholder `....` with the right code.

In [ ]:
# Trim outliers for DepDelay based on 1% and 90% percentiles
# Produce quantiles corresponding to 1% and 90%
depdelay_01pcntile <- df_flights %>%
  .... %>%
  ....

depdelay_90pcntile <- df_flights %>%
  .... %>%
  ....

# Print 1st and 90th quantiles respectively
cat(depdelay_01pcntile, "\n", depdelay_90pcntile)


Test your answer:



In [ ]:
. <- ottr::check("tests/Question 6.R")


Good job!!

Now that we have quantiles corresponding to 1% and 90%, let's filter the `df_flights` data to only include rows whose Departure delay falls within this range.

**Question 7.** Starting with the `df_flights` data, filter to only include rows whose **DepDelay** falls within 1st and 90th quantiles. Assign this to a variable name `df_flights`.

Fill in the placeholder `....` with the right code.

In [ ]:
# Filter data to remove outliers
df_flights <- df_flights %>%
  ....


Test your answer:



In [ ]:
. <- ottr::check("tests/Question 7.R")


You rock!!

Now, we can check the distribution of our two variables with outliers removed.


In [ ]:
# Distribution of DepDelay
show_distribution(var_data = select(df_flights, DepDelay), binwidth = 2)


In [ ]:
# Distribution of ArrDelay
show_distribution(var_data = select(df_flights, ArrDelay), binwidth = 2)


Much better!

Now that the data is all cleaned up, we can begin doing some exploratory analysis.

## **Explore the data**

Let's start with an overall view of the summary statistics for the numeric columns.


In [ ]:
# Obtain common summary statistics using summarytools package
df_flights %>%
  descr(stats = "common")


### **What are the mean departure and arrival delays?**

**Question 8.** Starting with the `df_flights` data, use `across()` within `summarize()` to find the mean across `DepDelay` and `ArrDelay` columns. Assign this to `df_delays` variable name. What are the mean delays?

Fill in the placeholder `....` with the right code.

In [ ]:
# Summarise the departure and arrival delays by finding the mean
df_delays <- df_flights %>%
  summarise(across(....))

df_delays


Test your answer:



In [ ]:
. <- ottr::check("tests/Question 8.R")


### **How do the carriers compare in terms of arrival delay performance?**

A box plot can be a good way for graphically depicting the distribution of groups of numerical data through their quantiles. The `geom` that takes care of box plots is `geom_boxplot`


In [ ]:
# Compare arrival delay across different carriers
df_flights %>%
  ggplot() +
  geom_boxplot(mapping = aes(x = Carrier, y = ArrDelay))


### **How do the carriers compare in terms of departure delay performance?**

Let's do the same for the departure delay performance.

We can also try and rearrange the `Carrier` levels in ascending order of the delay time and sprinkle some color to the plots too.


In [ ]:
df_flights %>%
  mutate(Carrier = fct_reorder(Carrier, DepDelay)) %>%
  ggplot() +
  geom_boxplot(mapping = aes(x = Carrier, y = DepDelay, color = Carrier),
  show.legend = FALSE)


Alternatively, to create the above plots, we can use `purr::map()` which allows us to apply a function to each column. See `?map` for more details.



In [ ]:
map(df_flights %>% select(ArrDelay, DepDelay), ~ ggplot(df_flights) +
  geom_boxplot(mapping = aes(x = Carrier, y = .x)) + ylab(""))


### **Are some days of the week more prone to arrival delays than others?**

Again, let's make use of a box plot to visually inspect the distribution of arrival delays depending on the day of the week. To successfully accomplish this, we have to first encode the week days to `categorical` variables, i.e. `factors` .


In [ ]:
# Encode day of the week as a categorical and make boxplots
df_flights %>%
  mutate(DayOfWeek = factor(DayOfWeek)) %>%
  ggplot() +
  geom_boxplot(mapping = aes(x = DayOfWeek, y = ArrDelay),
  show.legend = FALSE)


### **Are some days of the week more prone to departure delays than others?**

Now, over to you.

**Question 9.** Let's investigate whether some days of the week (x axis) are more prone to departure delays (y axis) than others. Start by encoding day of the week as a categorical variable.

Fill in the placeholder `....` with the right code.

In [ ]:
# Encode day of the week as a categorical variable
df_flights <- df_flights %>%
  mutate(....)

# Make a box plot of DayOfWeek and DepDelay
dep_delay_plot <- df_flights %>%
  ggplot() +
  geom_boxplot(mapping = aes(x = ...., y = ...., color = DayOfWeek),
  show.legend = FALSE) +
  scale_color_brewer(palette = "Dark2")

dep_delay_plot


What can you make out of this?
Test your answer:



In [ ]:
. <- ottr::check("tests/Question 9.R")


Great progress you are having!

### **Which departure airport has the highest average departure delay?**

To answer this, we have to first **group** the data **by** `OriginAirportName` and then **summarize** the observations by the **mean** of their Departure delay `DepDelay` and then **arrange** this in **descending** order of the mean departure delays.

Let's put this into code.


In [ ]:
# Use group_by %>% summarize to find airports with highest avg DepDelay
mean_departure_delays <- df_flights %>%
  group_by(OriginAirportName) %>%
  summarize(mean_dep_delay_time = mean(DepDelay)) %>%
  arrange(desc(mean_dep_delay_time))

# Print the first 7 rows
mean_departure_delays %>%
  slice_head(n = 7)


Fantastic!

Let's represent this using bar plots.


In [ ]:
mean_departure_delays %>%
  # Sort factor levels in descending order of delay time
  mutate(OriginAirportName = fct_reorder(OriginAirportName,
 desc(mean_dep_delay_time))) %>%
  ggplot() +
  geom_col(mapping = aes(x = OriginAirportName, y = mean_dep_delay_time),
 fill = "midnightblue", alpha = 0.7) +
  theme(
    # Rotate X markers so we can read them
    axis.text.x = element_text(angle = 90)
  )


Could you try and guess why Chicago Airport has most departure delay time or why Long Beach has the least?

### **Do late departures tend to result in longer arrival delays than on-time departures?**

**Question 10.** Starting with the `df_flights` data, first encode `DepDel15` column (a binary indicator that departure was delayed by more than 15 minutes) as categorical.

Use a **box plot** to investigate whether late departures (x-axis) tend to result in longer arrival delays (y-axis) than on-time departures. Map the fill aesthetic to the `DepDel15` variable.

> You can colour a box plot using either the `colour` aesthetic (like in the previous exercises), or, more usefully with the `fill` aesthetic.

Fill in the placeholder `....` with the right code.

In [ ]:
# Encode DepDel15 as a categorical variable
df_flights <- df_flights %>%
  mutate(DepDel15 = factor(DepDel15))

arr_delay_plot <- df_flights %>%
  ggplot() +
  geom_boxplot(mapping <- aes(x = ...., y = ...., fill = ....))

arr_delay_plot


Does this surprise you?
Test your answer:



In [ ]:
. <- ottr::check("tests/Question 10.R")


### **Which route (from origin airport to destination airport) has the most late arrivals?**

Finally, let's investigate travel routes. We'll start by adding a column `Route` that indicates the Origin and Destination airports.


In [ ]:
# Add a "Route" column
df_flights <- df_flights %>%
  mutate(Route = paste(OriginAirportName, DestAirportName, sep = ">"))


Great! Now we can use `group_by()`, `summarize()` and `arrange()` to find the routes with the most late arrivals



In [ ]:
# Make grouped summaries to find the total delay
# associated with a particular route
df_flights %>%
  group_by(Route) %>%
  summarize(ArrDel15 = sum(ArrDel15)) %>%
  arrange(desc(ArrDel15))


### **Which route has the highest average arrival delay time?**

Over to you!

**Question 11.** Starting with the `df_flights` data, group the observations by `Route` then create a summary tibble with a column name `ArrDelay` which represents the mean arrival delay time. Arrange this in descending order.

Assign your results to a variable name `df_route_arrdelay`.

Fill in the placeholder `....` with the right code.

In [ ]:
# Create grouped summaries of the arrival delay time
df_route_arrdelay <- df_flights %>%
  .... %>%
  summarise(ArrDelay = ....) %>%
  ....(desc(....))


# Print the first 5 rows
df_route_arrdelay %>%
  slice_head(n = 5)


Test your answer:



In [ ]:
. <- ottr::check("tests/Question 11.R")


Congratulations on finishing the first challenge! We'll wrap it at that for now. Of course there are other ways to approach this challenge. So please feel free to experiment, google and share your solutions with friends.

See you in the next module where we get started with Machine Learning!

Happy Learning,

[Eric,](https://twitter.com/ericntay) Gold Microsoft Learn Student Ambassador.
